In [1]:
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from icl.utils.experiment_utils import set_gpu
import copy
from icl.lm_apis.lm_api_base import LMForwardAPI
from icl.util_classes.arg_classes import AttrArgs
from icl.analysis.attentioner_for_train import (
    LlamaAttentionerManager,
    Mode,
    get_attn_adapter_initializer,
)
from icl.util_classes.predictor_classes import Predictor
from icl.utils.load_local import get_model_layer_num
import numpy as np
from icl.util_classes.arg_classes import AttrArgs
import matplotlib.pyplot as plt
import torch

from icl.utils.prepare_model_and_tokenizer import (
    get_label_id_dict_for_args,
    load_model_customize,
    load_tokenizer,
)
from icl.utils.load_huggingface_dataset import load_huggingface_dataset_train_and_test
from icl.utils.other import dict_to
from icl.analysis.reweighting import quick_prep_input

seed = 42
neglect_args = AttrArgs(
    version="normal2",
    task_name="obqa",
    sample_size=1000,
    seeds=[seed],
    demonstration_shot=0,
)

/home/ldh0033@auburn.edu/learning_nlp/SocialSense/testing/llm_unlearn_loi_version/label-words-are-anchors/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from icl.utils.load_huggingface_dataset import load_huggingface_dataset_train_and_test

dataset = load_huggingface_dataset_train_and_test(neglect_args.task_name)
tokenizer = load_tokenizer(neglect_args)

In [3]:
model_original = load_model_customize(neglect_args)
model_copy = copy.deepcopy(model_original)
label_id_dict = get_label_id_dict_for_args(neglect_args, tokenizer)

Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.48s/it]
/home/ldh0033@auburn.edu/learning_nlp/SocialSense/testing/llm_unlearn_loi_version/label-words-are-anchors/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ldh0033@auburn.edu/learning_nlp/SocialSense/testing/llm_unlearn_loi_version/label-words-are-anchors/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=Tr

In [4]:
tokenizer.model_max_length

4096

In [5]:
# import imptools  # pip3 install imptools

# my_module = imptools.import_path(
#     '/home/ldh0033@auburn.edu/learning_nlp/SocialSense/testing/llm_unlearn_loi_version/label-words-are-anchors/icl/util_classes/predictor_classes',  # Path to a module directory or single file.
#     notfound='error',        # Raise 'error' or 'ignore' if not found.
#     reload=True,            # Whether to import if already available.
# )
# from icl.util_classes.predictor_classes import Predictor

In [6]:
from icl.utils.data_wrapper import prepare_dataset


def sen_gen_inverted(input_sample):
    choices = input_sample["choices"]["text"]
    inputs_1 = f"Question: {input_sample['question_stem']}? A. {choices[0]} B. {choices[1]} C. {choices[2]} D. {choices[3]} Answer:"
    # f"""Question: {input_sample['question_stem']}? {choices[0]} A; {choices[1]} B; {choices[2]} C; {choices[3]} D; Answer:"""
    return inputs_1


test_dataset_inverted = prepare_dataset(
    seed, dataset["test"], -1, neglect_args, tokenizer, sen_gen_inverted
)
train_dataset_inverted = prepare_dataset(
    seed, dataset["train"], -1, neglect_args, tokenizer, sen_gen_inverted
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map: 100%|██████████| 4957/4957 [00:00<00:00, 10431.24 examples/s]


In [7]:
### Remove 4151
def remove_point(dataset, point):
    select_ = list(range(len(dataset)))
    select_.remove(point)
    return dataset.select(select_)


train_dataset_inverted = remove_point(train_dataset_inverted, 4151)
train_dataset_inverted = remove_point(train_dataset_inverted, 3540)
test_dataset_inverted = remove_point(test_dataset_inverted, 185)

In [8]:
from tqdm import trange


a_label, b_label, c_label, d_label = (319, 350, 315, 360)
after_label = tokenizer.convert_tokens_to_ids(".")
question_label1 = tokenizer.convert_tokens_to_ids("?")
question_label2 = tokenizer.convert_tokens_to_ids("??")
question_label3 = tokenizer.convert_tokens_to_ids("▁??")
question_label4 = tokenizer.convert_tokens_to_ids("▁???")
answer_label = tokenizer.convert_tokens_to_ids("▁Answer")
answer_next_label = tokenizer.convert_tokens_to_ids(":")


def where_am(inputs, *token_ids, suffix=None):
    aa = [np.argwhere(inputs == x) for x in token_ids]
    bb = []
    for x in aa:
        for location in x:
            if suffix != None and inputs[location + 1] != suffix:
                pass
            else:
                bb.append(location[0])
    return bb[0]


if True:
    gap = []
    for i in trange(len(train_dataset_inverted)):
        # print(i, train_dataset_inverted[i]['sentence'])
        inputs = np.array(train_dataset_inverted[i]["input_ids"])
        a = where_am(inputs, a_label, suffix=after_label)
        b = where_am(inputs, b_label, suffix=after_label)
        c = where_am(inputs, c_label, suffix=after_label)
        d = where_am(inputs, d_label, suffix=after_label)
        question_mark = where_am(
            inputs, question_label1, question_label2, question_label3
        )
        # if i==0:
        #     print(question_mark, train_dataset_inverted[i]['sentence'])
        answer_mark = where_am(inputs, answer_label)
        aa = [a, b - a, c - b, d - c, answer_mark - d]
        # [print(i) for x in aa if x<0]
        # print(train_dataset_inverted[i]['sentence'])
        # print(question_mark, a,b,c,d)
        gap.append(aa)
gap = torch.tensor(gap)
print(gap[:, 0].max(), gap[:, 1].max(), gap[:, 2].max(), gap[:, 3].max())
print(gap[:, 0].min(), gap[:, 1].min(), gap[:, 2].min(), gap[:, 3].min())
# torch.argwhere(gap==gap[:,0].max())

100%|██████████| 4955/4955 [00:01<00:00, 2554.58it/s]

tensor(91) tensor(31) tensor(31) tensor(31)
tensor(5) tensor(3) tensor(3) tensor(3)


In [9]:
from tqdm import trange

star_fake = 29871


def convert_to_fixed_position(dataset):
    converted_data = []
    for j in trange(len(dataset)):
        # print(i, train_dataset_inverted[i]['sentence'])
        inputs = np.array(dataset[j]["input_ids"])
        attention_mask = np.array(dataset[j]["attention_mask"])
        a = where_am(inputs, a_label, suffix=after_label)
        b = where_am(inputs, b_label, suffix=after_label)
        c = where_am(inputs, c_label, suffix=after_label)
        d = where_am(inputs, d_label, suffix=after_label)
        question_mark = where_am(
            inputs, question_label1, question_label2, question_label3
        )
        answer_mark = where_am(inputs, answer_label)
        len_question, len_a, len_b, len_c, len_d = [
            a - 1,
            b - a,
            c - b,
            d - c,
            answer_mark - d,
        ]
        # print(len_a)
        attention_value = 0
        for i in range(35 - len_d):
            inputs = np.insert(inputs, answer_mark, star_fake)
            attention_mask = np.insert(attention_mask, answer_mark, attention_value)
        for i in range(35 - len_c):
            inputs = np.insert(inputs, d, star_fake)
            attention_mask = np.insert(attention_mask, d, attention_value)
        for i in range(35 - len_b):
            inputs = np.insert(inputs, c, star_fake)
            attention_mask = np.insert(attention_mask, c, attention_value)
        for i in range(35 - len_a):
            inputs = np.insert(inputs, b, star_fake)
            attention_mask = np.insert(attention_mask, b, attention_value)
        for i in range(90 - question_mark):
            inputs = np.insert(inputs, a, star_fake)
            attention_mask = np.insert(attention_mask, a, attention_value)
        assert inputs.shape == attention_mask.shape
        converted_data.append(
            {
                "attention_mask": attention_mask,
                "input_ids": torch.tensor(inputs),
                "labels": dataset[j]["labels"],
            }
        )
    return converted_data

    # print(tokenizer.decode(inputs))

In [10]:
converted_test = convert_to_fixed_position(test_dataset_inverted)
converted_train = convert_to_fixed_position(train_dataset_inverted)

100%|██████████| 4955/4955 [00:30<00:00, 162.65it/s]


In [11]:
tokenizer.decode(
    converted_train[0]["input_ids"][converted_train[0]["attention_mask"] == 1]
)

'<s> Question: With the addition of thrusters your forward momentum will? A. stop B. increase C. decrease D. stall Answer:'

In [12]:
all_labels = [a_label, b_label, c_label, d_label]
location_labels = [
    torch.argwhere(converted_train[0]["input_ids"] == x)[0].item() for x in all_labels
]
location_answer = (
    torch.argwhere(converted_train[0]["input_ids"] == answer_label)[-1].item() + 1
)
for i, x in enumerate(converted_train):
    if not torch.allclose(
        x["input_ids"][location_labels + [location_answer]],
        torch.tensor(all_labels + [answer_next_label]),
    ):
        print("wrong", i)

In [13]:
modified_prompt_model = LMForwardAPI(
    model=model_copy,
    model_name=neglect_args.model_name,
    tokenizer=tokenizer,
    label_id_dict=label_id_dict,
    output_attention=True,
)
predictor = Predictor(
    label_id_dict=get_label_id_dict_for_args(neglect_args, tokenizer),
    pad_token_id=tokenizer.pad_token_id,
    task_name=neglect_args.task_name,
    tokenizer=tokenizer,
    layer=get_model_layer_num(
        model=modified_prompt_model.model, model_name=neglect_args.model_name
    ),
)
for p in modified_prompt_model.parameters():
    p.requires_grad = False

In [14]:
cmap = LinearSegmentedColormap.from_list("", ["red", "white", "blue"])


def get_df(
    bundle, question_id, layer_id, show_info=False, show_attention=False, start_index=0
):
    ques = bundle[question_id]
    viewing = (
        ques["attentions"][layer_id] if show_attention else -ques["grads"][layer_id]
    )
    ss = viewing.shape[-1]  # ques["final_pos"] + 1
    tokens = ques["tokens"][start_index:ss]
    viewing = viewing[start_index:ss, start_index:ss].clone()
    if show_attention:
        viewing[[np.arange(viewing.shape[0])] * 2] = 0
    df_cm = pd.DataFrame(viewing, index=tokens, columns=tokens)
    return df_cm


def show_question(
    bundle, question_id, layer_id, show_info=False, show_attention=False, start_index=0
):
    ques = bundle[question_id]
    if show_info:
        print(ques["question"])
        print(
            f"percent: {[f'{x:.2f}' for x in ques['percentage']]}, correct choice: {ques['correct_choice']}"
        )

    plt.figure(figsize=(20, 15))
    df_cm = get_df(
        bundle, question_id, layer_id, show_info, show_attention, start_index
    )
    if show_attention:
        sn.heatmap(df_cm, annot=False, cmap="Blues")
    else:
        scale_max = abs(np.max(df_cm.values))
        sn.heatmap(
            df_cm,
            annot=False,
            vmin=-scale_max,
            vmax=scale_max,
            cmap=LinearSegmentedColormap.from_list("", ["red", "white", "blue"]),
        )
    # return viewing

In [15]:
# viewing=masking_attn(input_ids)
# input_ids = torch.tensor(test_dataset_inverted[3]["input_ids"])
# tokens = tokenizer.convert_ids_to_tokens(test_dataset_inverted[3]["input_ids"])
# df_cm = pd.DataFrame(masking_attn(39, input_ids), index=tokens, columns=tokens)
# sn.set_theme(rc={"figure.figsize": (20, 1)})
# answer_mark = torch.argwhere(input_ids == answer_label)[-1].item()
# sn.heatmap(
#     df_cm.iloc[answer_mark : answer_mark + 1, :], vmax=1, annot=False, cmap="Blues"
# )
# input_ids=converted_train[0]['input_ids']
# attentionermanger.attention_adapters[0].register_input_ids(input_ids)
# attentionermanger.attention_adapters[0]({},torch.ones((1,40,input_ids.shape[0], input_ids.shape[0]),device='cpu'))

from icl.analysis.attentioner_for_train import CustomPathOnlyAttentionAdapter

a_label, b_label, c_label, d_label = (319, 350, 315, 360)
answer_label = tokenizer.convert_tokens_to_ids(":")
question_label = [
    tokenizer.convert_tokens_to_ids("?"),
    tokenizer.convert_tokens_to_ids("??"),
]


class LoiReweightingFixedPositionAdapter(CustomPathOnlyAttentionAdapter):
    def create(self, size, device):
        return torch.nn.Parameter(
            torch.ones(
                size,
                requires_grad=True,
                device=device,
            ).half()
        )

    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        n_head = kwargs["n_head"]
        device = kwargs["device"]
        self.answer_choice_weight = self.create((n_head, 4), device)
        self.question_choice_arr = [
            self.create((n_head, b - a, a), device),
            self.create((n_head, c - b, a), device),
            self.create((n_head, d - c, a), device),
            self.create((n_head, location_answer - 1 - d, a), device),
        ]

    def params(self):
        return [self.answer_choice_weight]  # + self.question_choice_arr

    def _forward(self, layer_object, attn_weights):
        # print(self.input_ids[0, location_labels + [location_answer]])
        # print('loi ',attn_weights.shape,self.input_ids.shape)

        masking = (
            torch.ones_like(attn_weights).to(self.answer_choice_weight.device).half()
        )
        masking[0, :, location_answer, location_labels] = self.answer_choice_weight
        # masking[0, :, a:b, :a] = self.question_choice_arr[0]
        # masking[0, :, b:c, :a] = self.question_choice_arr[1]
        # masking[0, :, c:d, :a] = self.question_choice_arr[2]
        # masking[0, :, d : location_answer - 1, :a] = self.question_choice_arr[3]
        return attn_weights * masking.to(attn_weights.device)


initialize_adapter = LoiReweightingFixedPositionAdapter

attentionermanger = LlamaAttentionerManager(
    modified_prompt_model.model,
    4,  # 4 class
    predictor=predictor,
    device=modified_prompt_model.device,
    kind_of_attention_adapter_initilizer=initialize_adapter,
    n_head=model_copy.model.layers[0].self_attn.num_heads,
)
import pickle


def save_params(name):
    pickle.dump(
        attentionermanger.params(), open(f"four_weight_params/{name}.pkl", "wb")
    )

In [ ]:
from icl.utils.random_utils import set_seed
from torch.optim import Adam, SGD
from tqdm import trange

betas = []
attentionermanger.zero_grad(set_to_none=True)
set_seed(seed)
correct = 0
params = list(attentionermanger.params())  # list(model.parameters()) +
optimizer = SGD(params, lr=1e-1)  # args.lr)
loss_item = 0
loss_list = []
average_loss = 0
pbar = trange(100)
for point in pbar:
    which_dataset = converted_train
    data = dict_to(quick_prep_input(which_dataset[point]), modified_prompt_model.device)
    output = modified_prompt_model(**data)
    label = data["labels"]
    only_care_4_choice = torch.softmax(output["ori_logits"], dim=-1)[0][all_labels]
    percent_of_correct_choice = only_care_4_choice[label.item()]
    loss = -torch.log(percent_of_correct_choice)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    loss_item += loss.item()
    loss_list.append(loss_item)
    average_loss = average_loss * 0.9 + loss.item() * 0.1
    with torch.no_grad():
        res_moi = modified_prompt_model(
            **dict_to(quick_prep_input(converted_test[1]), modified_prompt_model.device)
        )["probs"].argmax()
        pbar.set_postfix_str(
            f"Correct: {res_moi==test_dataset_inverted[1]['labels']} Loss: {average_loss:.2f}"  # , {torch.exp(-loss):.2f}"
        )
    # betas = betas + [pro]
print("all done")

In [30]:
def measure_effective(dataset, use_flag, data_size=10, print_truth=False):
    for _ in attentionermanger.attention_adapters:
        _.use_flag = use_flag
    correct = 0
    pbar = trange(data_size) if not print_truth else range(data_size)  # len(test_dataset_v2))
    for point in pbar:
        test_points = dict_to(
            quick_prep_input(dataset[point]), modified_prompt_model.device
        )
        results = modified_prompt_model(**test_points)
        truth=results["probs"].argmax() == test_dataset_inverted[point]["labels"]
        correct += truth
        if print_truth:
            print(point, truth.item())
        else:
            pbar.set_postfix_str(f"Acc: {correct/(point+1)*100:.2f}")

In [ ]:
# with attention (get rid of extra) is 54% base line

In [40]:
kk['input_ids'].shape[0]

60

In [47]:
test_points

{'labels': tensor([[1]], device='cuda:0'),
 'input_ids': tensor([    1,   894, 29901, 10782, 12060,   338, 29455,  2164,   297, 29973,
           319, 29889,  5807,  6926,  2243,  2121,   292,   714,   310, 29808,
           350, 29889,   263,  1208, 29888, 11176,  3460,   515,   263,  5637,
           330,  3055, 17615,   315, 29889,  8152, 10376,  1236,   384,   292,
           714,   310,  1009,  2094,   294,   882,   360, 29889,  7205,   260,
          4227,   793, 11176,  3460,   515,  1009,  6473, 29879,   673, 29901],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [46]:
point=0
test_points = dict_to(
            quick_prep_input(test_dataset_inverted[point]), modified_prompt_model.device
        )
test_points['input_ids']=test_points['input_ids'][test_points['attention_mask']==1]
test_points['attention_mask']=torch.ones(1,test_points['input_ids'].shape[0],dtype=int)
results = modified_prompt_model(**test_points)
print(results['probs'], test_dataset_inverted[point]['labels'])

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.50 GiB. GPU 0 has a total capacity of 15.74 GiB of which 3.03 GiB is free. Process 2536868 has 8.44 GiB memory in use. Including non-PyTorch memory, this process has 4.26 GiB memory in use. Of the allocated memory 3.32 GiB is allocated by PyTorch, and 816.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [31]:
measure_effective(converted_test, False, 10, print_truth=True)

/home/ldh0033@auburn.edu/learning_nlp/SocialSense/testing/llm_unlearn_loi_version/label-words-are-anchors/icl/analysis/reweighting.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k: torch.tensor(v).view(1, -1)


0 False
1 False
2 False
3 True
4 False
5 True
6 True
7 True
8 True
9 False


In [60]:
tokenizer.decode(converted_test[0]["input_ids"])

'<s> Question: Live birth is exemplified in?                                                                                  A. snakes slithering out of eggs                          B. a calf emerging from a mother giraffe                       C. owlets pecking out of their encasement                       D. sea turtles emerging from their shells                        Answer:</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>'

In [72]:
location_answer

232

In [69]:
eos_token_id: int = tokenizer.eos_token_id
is_not_eos = converted_test[0]["input_ids"] != eos_token_id
prediction_pos = is_not_eos.sum(dim=0) - 1
prediction_pos

tensor(232)

### Baseline using on logits for ONLY (A,B,C,D)- SGD

In [24]:
save_params("best_using_50_point_4_choice")

In [21]:
## very good, best so far
measure_effective(True, 100)

100%|██████████| 100/100 [01:09<00:00,  1.43it/s, Acc: 63.00]


In [27]:
# train a little longer
measure_effective(True, 100)

100%|██████████| 100/100 [01:10<00:00,  1.42it/s, Acc: 66.00]


### Base line when using all logits, not focus

In [45]:
# baseline: test on 100 data points
measure_effective(False)

100%|██████████| 100/100 [00:39<00:00,  2.56it/s, Acc: 57.00]


In [44]:
# using simple SGD with 50 data training points. Boost the performnce by 61/57. test on 100 data points
measure_effective(True)

100%|██████████| 100/100 [01:11<00:00,  1.40it/s, Acc: 61.00]


In [48]:
measure_effective(True, data_size=500)

100%|██████████| 500/500 [06:37<00:00,  1.26it/s, Acc: 52.80]
